In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import psycopg2
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from shapely.geometry import MultiPolygon
from zipfile import ZipFile
import requests 
import sys

In [2]:
import yaml

with open('../../config/postgres.yaml') as f:
    engine_configs = yaml.load(f, Loader=yaml.FullLoader)
    
try:
    engine = create_engine('postgresql://{username}:{password}@{host}:{port}/{dbname}'.format(**engine_configs))
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [3]:
def process_geometry_SQL_insert(gdf):
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement((MultiPolygon([x]) if x.geom_type == 'Polygon' else x).wkt, srid=4326))
    gdf = gdf.drop('geometry', 1)
    return gdf

In [4]:
tracts_gdf = gpd.read_file('zip://../../data/LA/mobile-phone/census_tracts.zip')
tracts_gdf = tracts_gdf[['geometry', 'GEOID']]
tracts_gdf.head()

,geometry,GEOID
0,"POLYGON ((-118.30229 34.25870, -118.30079 34.2...",06037101110
1,"POLYGON ((-118.30333 34.27354, -118.30318 34.2...",06037101122
2,"POLYGON ((-118.29945 34.25598, -118.28592 34.2...",06037101210
3,"POLYGON ((-118.28592 34.24896, -118.28592 34.2...",06037101220
4,"POLYGON ((-118.27247 34.23253, -118.27194 34.2...",06037101300


In [5]:
sql = """
SELECT b.original_id, bid, sp_id
FROM blocks_group b
INNER JOIN spatial_groups as sp on b.bid = ANY(sp.lower_ids)
WHERE b.city='LA'
"""

blocks2spid_df = pd.read_sql(sql, engine)
blocks2spid_df['GEOID'] = blocks2spid_df['original_id'].str[0:11]
blocks2spid_df.head()

,original_id,bid,sp_id,GEOID
0,060371393012,452172,343088,06037139301
1,060371375012,452185,343088,06037137501
2,060372403002,452080,343080,06037240300
3,060371320011,452040,343071,06037132001
4,060371394021,456434,343088,06037139402


In [6]:
njoins_df = blocks2spid_df[['bid', 'GEOID']].drop_duplicates().groupby('GEOID').size().to_frame('count').reset_index()
njoins_df.head()

,GEOID,count
0,06037101110,3
1,06037101122,2
2,06037101210,2
3,06037101220,2
4,06037101300,4


In [7]:
blocks2spid_df = pd.merge(blocks2spid_df, njoins_df, on='GEOID')
blocks2spid_df.head()

,original_id,bid,sp_id,GEOID,count
0,060371393012,452172,343088,06037139301,3
1,060371393013,459744,343088,06037139301,3
2,060371393011,465885,343088,06037139301,3
3,060371393013,459744,343073,06037139301,3
4,060371393012,452172,343091,06037139301,3


In [8]:
blocks2spid_df[blocks2spid_df.GEOID=='06037980028']

,original_id,bid,sp_id,GEOID,count
11656,060379800281,472486,343109,06037980028,1
11657,060379800281,472486,343455,06037980028,1
11658,060379800281,472486,343737,06037980028,1
11659,060379800281,472486,343749,06037980028,1
11660,060379800281,472486,343928,06037980028,1
11661,060379800281,472486,344042,06037980028,1
11662,060379800281,472486,344151,06037980028,1
11663,060379800281,472486,344326,06037980028,1
11664,060379800281,472486,344998,06037980028,1
11665,060379800281,472486,345036,06037980028,1


In [9]:
blocks2spid_unique_df = blocks2spid_df.drop_duplicates(subset=['GEOID', 'sp_id'])[['GEOID', 'sp_id', 'count']]
blocks2spid_unique_df.head()

,GEOID,sp_id,count
0,06037139301,343088,3
3,06037139301,343073,3
4,06037139301,343091,3
6,06037139301,343153,3
7,06037139301,343258,3


In [10]:
blocks2spid_unique_df[blocks2spid_unique_df.GEOID == '06037262302']

,GEOID,sp_id,count
32276,06037262302,343315,2
32277,06037262302,343631,2
32278,06037262302,343853,2
32280,06037262302,344492,2
32282,06037262302,344544,2
32284,06037262302,344702,2
32285,06037262302,344765,2
32286,06037262302,345125,2
32287,06037262302,345282,2
32288,06037262302,345445,2


## ODs

In [11]:
zip_file = ZipFile('../../data/LA/mobile-phone/travel_demand_LA.zip')
zip_file.infolist()

[<ZipInfo filename='travel_demand_LA.csv' compress_type=deflate filemode='-rw-rw-r--' file_size=991736528 compress_size=149183206>]

In [12]:
types = {str(x): np.float32 for x in range(0,24)}
types['O_Tract'] = str
types['D_Tract'] = str
types['HBW'] = np.float32
types['HBO'] = np.float32
types['NHB'] = np.float32
types['lon1'] = np.float32
types['lat1'] = np.float32
types['lon2'] = np.float32
types['lat2'] = np.float32

travel_df = pd.read_csv(zip_file.open('travel_demand_LA.csv'), dtype=types)
travel_df = travel_df.drop(['lon1', 'lat1', 'lon2', 'lat2'], axis=1)
travel_df['tot'] = travel_df[[str(x) for x in range(0,24)]].sum(axis=1)
travel_df = travel_df[['O_Tract', 'D_Tract', 'HBW', 'HBO', 'NHB', 'tot']]
travel_df.head()

,O_Tract,D_Tract,HBW,HBO,NHB,tot
0,06037406200,06037264102,0.0,0.0,0.0,0.0
1,06037535101,06037570702,0.0,0.0,0.0,0.0
2,06037141202,06037139401,0.0,0.0,0.0,0.0
3,06037500600,06037601802,0.0,0.0,0.0,0.0
4,06037195100,06037405701,0.0,0.0,0.0,0.0


In [13]:
travel_df = travel_df[['O_Tract', 'D_Tract', 'HBW', 'HBO', 'NHB', 'tot']]

In [14]:
travel_df = travel_df[(travel_df['HBW']!=0) | (travel_df['HBO']!=0) | (travel_df['NHB']!=0) | (travel_df['tot']!=0)]
travel_df.head()

,O_Tract,D_Tract,HBW,HBO,NHB,tot
14,06037703100,06037651304,0.0,20.0,0.0,20.0
30,06037141400,06037800327,0.0,0.0,20.0,20.0
42,06037206200,06037212203,0.0,20.0,40.0,60.0
46,06037534501,06037534900,0.0,20.0,0.0,20.0
50,06037402600,06037407802,0.0,20.0,0.0,20.0


In [15]:
# Barrios to blockgroup
od_sp_groups_df = pd.merge(travel_df[['O_Tract', 'D_Tract', 'HBO', 'NHB', 'tot']], blocks2spid_unique_df.rename(columns={'sp_id': 'o_sp_id'}), left_on='O_Tract', right_on='GEOID').drop(['GEOID'], axis=1)
od_sp_groups_df.loc[:, 'tot'] = od_sp_groups_df['tot'] / od_sp_groups_df['count']
od_sp_groups_df = od_sp_groups_df.drop(['count', 'O_Tract'], axis=1)
od_sp_groups_df = od_sp_groups_df.groupby(['o_sp_id', 'D_Tract'], as_index=False).sum()
od_sp_groups_df.head()

,o_sp_id,D_Tract,HBO,NHB,tot
0,343071,06037103200,20.0,0.0,6.666667
1,343071,06037103300,20.0,0.0,6.666667
2,343071,06037104105,40.0,0.0,13.333333
3,343071,06037104108,20.0,20.0,11.666667
4,343071,06037104124,20.0,20.0,11.666667


In [16]:
od_sp_groups_df = pd.merge(od_sp_groups_df, blocks2spid_unique_df.rename(columns={'sp_id': 'd_sp_id'}), left_on='D_Tract', right_on='GEOID').drop(['GEOID'], axis=1)
od_sp_groups_df.loc[:, 'tot'] = od_sp_groups_df['tot'] / od_sp_groups_df['count']
od_sp_groups_df = od_sp_groups_df.drop(['count', 'D_Tract'], axis=1)
od_sp_groups_df = od_sp_groups_df.groupby(['o_sp_id', 'd_sp_id'], as_index=False).sum()
od_sp_groups_df.head()

,o_sp_id,d_sp_id,HBO,NHB,tot
0,343071,343071,900.0,640.0,159.722222
1,343071,343073,820.0,280.0,148.333333
2,343071,343075,40.0,0.0,10.833333
3,343071,343076,100.0,0.0,18.055556
4,343071,343077,20.0,0.0,3.333333


In [17]:
all_sp_ids = sorted([str(x) for x in list(set(blocks2spid_df.sp_id.values))])

### Fix missing links

In [18]:
import itertools
tuples = list(itertools.product(all_sp_ids, all_sp_ids))

od_sp_groups_df['o_sp_id'] = od_sp_groups_df['o_sp_id'].astype(str)
od_sp_groups_df['d_sp_id'] = od_sp_groups_df['d_sp_id'].astype(str)
od_sp_groups_df = od_sp_groups_df.set_index(['o_sp_id', 'd_sp_id']).reindex(tuples).fillna(0).reset_index()
od_sp_groups_df.head()

,o_sp_id,d_sp_id,HBO,NHB,tot
0,343071,343071,900.0,640.0,159.722222
1,343071,343072,0.0,0.0,0.000000
2,343071,343073,820.0,280.0,148.333333
3,343071,343074,0.0,0.0,0.000000
4,343071,343075,40.0,0.0,10.833333


In [19]:
#Tot 0 ?
od_sp_groups_df[od_sp_groups_df.tot == 0].head()

,o_sp_id,d_sp_id,HBO,NHB,tot
1,343071,343072,0.0,0.0,0.0
3,343071,343074,0.0,0.0,0.0
10,343071,343081,0.0,0.0,0.0
15,343071,343086,0.0,0.0,0.0
23,343071,343094,0.0,0.0,0.0


## Extras

In [20]:
od_extra_df = pd.merge(travel_df[['O_Tract', 'D_Tract', 'HBO', 'NHB', 'tot']], blocks2spid_unique_df.rename(columns={'sp_id': 'o_sp_id'}), left_on='O_Tract', right_on='GEOID', how='left').drop(['GEOID'], axis=1)
od_extra_df = pd.merge(od_extra_df, blocks2spid_unique_df.rename(columns={'sp_id': 'd_sp_id'}), left_on='D_Tract', right_on='GEOID', how='left').drop(['GEOID'], axis=1)
od_extra_df.head()

,O_Tract,D_Tract,HBO,NHB,tot,o_sp_id,count_x,d_sp_id,count_y
0,06037703100,06037651304,20.0,0.0,20.0,343242.0,6.0,NaN,NaN
1,06037703100,06037651304,20.0,0.0,20.0,343456.0,6.0,NaN,NaN
2,06037703100,06037651304,20.0,0.0,20.0,343817.0,6.0,NaN,NaN
3,06037703100,06037651304,20.0,0.0,20.0,343879.0,6.0,NaN,NaN
4,06037703100,06037651304,20.0,0.0,20.0,343914.0,6.0,NaN,NaN


In [21]:
od_extra_df = od_extra_df[(od_extra_df.o_sp_id.isnull() & ~od_extra_df.d_sp_id.isnull()) | ((~od_extra_df.o_sp_id.isnull()) & od_extra_df.d_sp_id.isnull())]

# from out to LA
in_extra_df = od_extra_df[od_extra_df.o_sp_id.isnull()].groupby('d_sp_id', as_index=False).sum()
in_extra_df['d_sp_id'] = in_extra_df['d_sp_id'].astype(int).astype(str)
in_extra_df = in_extra_df[['d_sp_id', 'HBO', 'NHB', 'tot']]
in_extra_df['ntrips'] = in_extra_df['tot'] 
in_extra_df.head()

,d_sp_id,HBO,NHB,tot,ntrips
0,343071,1840.0,1220.0,3560.0,3560.0
1,343072,10980.0,6840.0,21820.0,21820.0
2,343073,3400.0,2260.0,6460.0,6460.0
3,343074,7240.0,4580.0,13040.0,13040.0
4,343075,8300.0,5680.0,16080.0,16080.0


In [22]:
# from LA to out
out_extra_df = od_extra_df[od_extra_df.d_sp_id.isnull()].groupby('o_sp_id', as_index=False).sum()
out_extra_df['o_sp_id'] = out_extra_df['o_sp_id'].astype(int).astype(str)
out_extra_df = out_extra_df[['o_sp_id', 'HBO', 'NHB', 'tot']]
out_extra_df['ntrips'] = out_extra_df['tot'] 
out_extra_df.head()

,o_sp_id,HBO,NHB,tot,ntrips
0,343071,1760.0,1100.0,3420.0,3420.0
1,343072,11540.0,8160.0,22320.0,22320.0
2,343073,3220.0,2360.0,6280.0,6280.0
3,343074,8400.0,3000.0,14140.0,14140.0
4,343075,8460.0,4080.0,16360.0,16360.0


## Blocks_attract

In [23]:
blocks2bid_unique_df = blocks2spid_df.drop_duplicates(subset=['bid'])[['bid', 'GEOID', 'count']]
blocks2bid_unique_df.head()

,bid,GEOID,count
0,452172,06037139301,3
1,459744,06037139301,3
2,465885,06037139301,3
32,452185,06037137501,2
34,466081,06037137501,2


In [24]:
od_bid_groups_df = pd.merge(travel_df[['O_Tract', 'D_Tract', 'HBO', 'NHB', 'tot']], blocks2bid_unique_df.rename(columns={'bid': 'o_bid'}), left_on='O_Tract', right_on='GEOID').drop(['GEOID'], axis=1)
od_bid_groups_df.loc[:, 'tot'] = od_bid_groups_df['tot'] / od_bid_groups_df['count']
od_bid_groups_df = od_bid_groups_df.drop(['count', 'O_Tract'], axis=1)
od_bid_groups_df = od_bid_groups_df.groupby(['o_bid', 'D_Tract'], as_index=False).sum()

od_bid_groups_df = pd.merge(od_bid_groups_df, blocks2bid_unique_df.rename(columns={'bid': 'd_bid'}), left_on='D_Tract', right_on='GEOID').drop(['GEOID'], axis=1)
od_bid_groups_df.loc[:, 'tot'] = od_bid_groups_df['tot'] / od_bid_groups_df['count']
od_bid_groups_df = od_bid_groups_df.drop(['count', 'D_Tract'], axis=1)
od_bid_groups_df = od_bid_groups_df.groupby(['o_bid', 'd_bid'], as_index=False).sum()

#od_bid_groups_df = od_bid_groups_df.set_index('d_bid')

od_bid_groups_df.head()

,o_bid,d_bid,HBO,NHB,tot
0,452040,452040,40.0,20.0,6.666667
1,452040,452062,40.0,0.0,4.444444
2,452040,452072,0.0,20.0,1.111111
3,452040,452130,20.0,0.0,3.333333
4,452040,452172,0.0,20.0,2.222222


In [25]:
sql = """
SELECT sp_id::text, unnest(lower_ids)::text as bid FROM spatial_groups where city='LA'
"""

blocks_spatial_df = pd.read_sql(sql, engine)
blocks_spatial_df.head()

,sp_id,bid
0,343071,453306
1,343071,467996
2,343071,468050
3,343071,458067
4,343071,469727


In [26]:
attract_df = od_sp_groups_df[['o_sp_id']].drop_duplicates().set_index('o_sp_id')

attract_df['attract'] = 0.
for i, spid in enumerate(attract_df.index.values):
    bids = blocks_spatial_df[blocks_spatial_df.sp_id == spid]['bid'].values
    s = od_bid_groups_df[(od_bid_groups_df.d_bid.isin(bids)) & (~(od_bid_groups_df.o_bid.isin(bids)))]['NHB'].sum()
    attract_df.loc[spid, 'attract'] = s

attract_df = attract_df.reset_index()    
attract_df.head()

,o_sp_id,attract
0,343071,53640.0
1,343072,91300.0
2,343073,80340.0
3,343074,41720.0
4,343075,106140.0


### Save "other" trips to out and to in

In [27]:
trips_other = od_sp_groups_df[['o_sp_id', 'd_sp_id', 'tot', 'NHB']].copy() #[od_sp_groups_df.o_sp_id == od_sp_groups_df.d_sp_id]
trips_other['ntrips'] = trips_other['tot'] 
trips_other = trips_other.drop(['tot'], axis=1)
trips_other.head()

,o_sp_id,d_sp_id,NHB,ntrips
0,343071,343071,640.0,159.722222
1,343071,343072,0.0,0.000000
2,343071,343073,280.0,148.333333
3,343071,343074,0.0,0.000000
4,343071,343075,0.0,10.833333


In [28]:
trips_attract = trips_other[trips_other.o_sp_id != trips_other.d_sp_id].copy()
trips_attract = trips_attract.rename(columns={'NHB': 'attract'}).drop('o_sp_id', axis=1)
trips_attract = trips_attract.groupby('d_sp_id', as_index=False).sum()
trips_attract = trips_attract.rename(columns={'d_sp_id': 'o_sp_id'}).drop('ntrips', axis=1)
trips_attract.head()

,o_sp_id,attract
0,343071,151760.0
1,343072,322880.0
2,343073,273160.0
3,343074,167040.0
4,343075,437240.0


In [29]:
trips_attract = pd.concat((trips_attract, in_extra_df.rename(columns={'d_sp_id': 'o_sp_id', 'NHB': 'attract'})[['o_sp_id', 'attract']]))
trips_attract = trips_attract.groupby('o_sp_id', as_index=False).sum()
trips_attract.head()

,o_sp_id,attract
0,343071,152980.0
1,343072,329720.0
2,343073,275420.0
3,343074,171620.0
4,343075,442920.0


In [30]:
trips_attract = attract_df.reset_index()[['o_sp_id', 'attract']]

In [31]:
trips_out = trips_other[trips_other.o_sp_id != trips_other.d_sp_id][['o_sp_id', 'NHB', 'ntrips']]
trips_out = pd.concat((trips_out, out_extra_df[['o_sp_id', 'NHB', 'ntrips']]))
trips_out = trips_out.groupby('o_sp_id', as_index=False).sum()
trips_out = trips_out.rename(columns={'ntrips': 'nout'})
trips_out = trips_out.drop(['NHB'], axis=1)
trips_out.head()

,o_sp_id,nout
0,343071,71788.345238
1,343072,188961.000000
2,343073,159490.055556
3,343074,98340.833333
4,343075,320766.500000


In [32]:
trips_in = trips_other[trips_other.o_sp_id == trips_other.d_sp_id].groupby('o_sp_id', as_index=False).sum()
trips_in = trips_in.rename(columns={'ntrips': 'nin'})
trips_in = trips_in.drop(['NHB'], axis=1)
trips_in.head()

,o_sp_id,nin
0,343071,159.722222
1,343072,566.111111
2,343073,525.000000
3,343074,344.444444
4,343075,1162.777778


In [33]:
df_all = pd.merge(trips_in, trips_out, on='o_sp_id')
df_all = pd.merge(trips_attract, df_all, on='o_sp_id')
df_all.head()

,o_sp_id,attract,nin,nout
0,343071,53640.0,159.722222,71788.345238
1,343072,91300.0,566.111111,188961.000000
2,343073,80340.0,525.000000,159490.055556
3,343074,41720.0,344.444444,98340.833333
4,343075,106140.0,1162.777778,320766.500000


In [34]:
df_all = pd.merge(trips_in, trips_out, on='o_sp_id')
df_all = pd.merge(trips_attract, df_all, on='o_sp_id')
df_all.head()

,o_sp_id,attract,nin,nout
0,343071,53640.0,159.722222,71788.345238
1,343072,91300.0,566.111111,188961.000000
2,343073,80340.0,525.000000,159490.055556
3,343074,41720.0,344.444444,98340.833333
4,343075,106140.0,1162.777778,320766.500000


In [35]:
df_all.to_sql('temptable3', engine, if_exists='replace', index=False)

In [38]:
sql = """
INSERT INTO spatial_groups_trips (sp_id, city, spatial_name, num_Otrips_in, num_Otrips_out, attract) 
SELECT c.o_sp_id::int, 'LA', 'ego', c.nin, c.nout, c.attract
FROM temptable3 c 
"""

result = engine.execute(text(sql))

### Save OD

In [23]:
ODs_matrix_df = od_sp_groups_df.copy()
ODs_matrix_df = ODs_matrix_df.pivot(index='o_sp_id', columns='d_sp_id', values='tot')
ODs_matrix_df.head()

d_sp_id,181963,181964,181965,181966,181967,181968,181969,181970,181971,181972,...,184460,184461,184462,184463,184464,184465,184466,184467,184468,184469
o_sp_id,,,,,,,,,,,,,,,,,,,,,
181963,77.777778,13.888889,72.222222,68.888889,102.222222,31.333333,11.111111,60.000000,146.666667,5.777778,...,11.111111,6.666667,18.888889,6.666667,6.666667,11.111111,20.000000,30.000000,13.333333,6.666667
181964,13.888889,84.027778,29.722222,48.888889,17.777778,16.666667,60.000000,7.222222,58.333333,3.888889,...,5.000000,15.000000,17.222222,18.055556,11.666667,5.000000,20.000000,15.833333,18.333333,15.000000
181965,54.444444,30.000000,297.777778,394.444444,70.555556,75.000000,42.222222,40.000000,213.333333,17.777778,...,0.000000,20.000000,0.000000,10.000000,23.333333,10.000000,33.333333,0.000000,18.333333,30.000000
181966,57.777778,69.166667,437.777778,544.444444,72.222222,81.333333,102.222222,30.000000,331.666667,19.111111,...,3.333333,13.333333,0.000000,15.000000,25.000000,13.333333,58.333333,0.000000,33.333333,23.333333
181967,120.000000,16.944444,62.777778,62.777778,222.222222,48.888889,27.777778,131.111111,290.000000,3.333333,...,4.444444,0.000000,18.333333,10.000000,3.333333,11.111111,13.333333,22.777778,10.000000,6.666667


In [24]:
ODs_matrix_df['city'] = 'LA'

In [25]:
ODs_matrix_df.to_csv('../../data/generated_files/LA_ODs.csv')

### Ambient population

In [5]:
sql = """
SELECT b.original_id, bid, sp_id
FROM blocks_group b
INNER JOIN spatial_groups as sp on b.bid = sp.core_id
WHERE b.city='LA'
"""

blocks2coreid_df = pd.read_sql(sql, engine)
blocks2coreid_df['GEOID'] = blocks2coreid_df['original_id'].str[0:11]
blocks2coreid_df.head()

,original_id,bid,sp_id,GEOID
0,060372753113,235767,182084,06037275311
1,060372270202,238114,184431,06037227020
2,060372932022,235652,181969,06037293202
3,060372941203,235653,181970,06037294120
4,060371892012,236061,182378,06037189201


In [9]:
njoins_coreid_df = blocks2coreid_df[['bid', 'GEOID']].drop_duplicates().groupby('GEOID').size().to_frame('count').reset_index()
njoins_coreid_df.head()

,GEOID,count
0,06037101110,3
1,06037101122,2
2,06037101210,2
3,06037101220,2
4,06037101300,4


In [10]:
blocks2coreid_df = pd.merge(blocks2coreid_df, njoins_coreid_df, on='GEOID')
blocks2coreid_df.head()

,original_id,bid,sp_id,GEOID,count
0,060372753113,235767,182084,06037275311,4
1,060372753112,235780,182097,06037275311,4
2,060372753114,235827,182144,06037275311,4
3,060372753111,235915,182232,06037275311,4
4,060372270202,238114,184431,06037227020,2


In [11]:
ambient_df = pd.read_csv('../../data/LA/mobile-phone/hourly_stay_LA.csv', dtype={'tract': str})
ambient_df[ambient_df.tract == '06037101110'].head()

,tract,0,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,lon,lat
869,06037101110,5002.0,4954.0,4924.0,4900.0,4874.0,4834.0,4716.0,4330.0,3606.0,...,3612.0,3886.0,4212.0,4314.0,4454.0,4420.0,4414.0,4410.0,-118.292987,34.259474


In [12]:
blocks2coreid_unique_df = blocks2coreid_df.drop_duplicates(subset=['GEOID', 'sp_id'])[['GEOID', 'sp_id', 'count']]
blocks2coreid_unique_df.head()

,GEOID,sp_id,count
0,06037275311,182084,4
1,06037275311,182097,4
2,06037275311,182144,4
3,06037275311,182232,4
4,06037227020,184431,2


In [13]:
ambient_sp_id_df = pd.merge(ambient_df, blocks2coreid_df[['GEOID', 'count', 'bid']].drop_duplicates(subset=['GEOID', 'bid', 'count']).rename(columns={'GEOID': 'tract'}), on='tract')

columns = [str(x) for x in range(0,24)]
for c in columns:
    ambient_sp_id_df.loc[:, c] = ambient_sp_id_df.loc[:, c]/ambient_sp_id_df['count']

ambient_sp_id_df.head()

,tract,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,lon,lat,count,bid
0,06037183810,1175.0,1169.0,1151.0,1150.0,1146.0,1137.0,1148.0,1209.0,1213.0,...,1153.0,1173.0,1165.0,1160.0,1152.0,1139.0,-118.197850,34.107365,2,237227
1,06037183810,1175.0,1169.0,1151.0,1150.0,1146.0,1137.0,1148.0,1209.0,1213.0,...,1153.0,1173.0,1165.0,1160.0,1152.0,1139.0,-118.197850,34.107365,2,237296
2,06037137301,2138.0,2126.0,2104.0,2106.0,2096.0,2098.0,2100.0,2094.0,2084.0,...,1894.0,1880.0,1874.0,1930.0,1960.0,1936.0,-118.634042,34.176253,1,237157
3,06037137302,2289.0,2265.0,2241.0,2234.0,2229.0,2227.0,2206.0,2003.0,1912.0,...,1868.0,1883.0,1875.0,1860.0,1999.0,1985.0,-118.654196,34.177348,2,237098
4,06037137302,2289.0,2265.0,2241.0,2234.0,2229.0,2227.0,2206.0,2003.0,1912.0,...,1868.0,1883.0,1875.0,1860.0,1999.0,1985.0,-118.654196,34.177348,2,237204


In [14]:
ambient_sp_id_df = ambient_sp_id_df.groupby('bid', as_index=False).sum()
ambient_sp_id_df['ambient_avg'] = ambient_sp_id_df[[str(x) for x in range(0,24)]].mean(axis=1)
ambient_sp_id_df.head()

,bid,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,lon,lat,count,ambient_avg
0,235646,989.333333,982.666667,978.666667,969.333333,967.333333,966.666667,984.666667,995.333333,1044.666667,...,1003.333333,986.000000,966.000000,959.333333,948.0,940.666667,-118.260391,33.797993,3,1009.055556
1,235647,983.000000,983.000000,986.000000,984.000000,987.000000,995.500000,1080.500000,1270.500000,1629.000000,...,1274.500000,1194.500000,1183.000000,1170.000000,1144.5,1100.000000,-118.302154,33.848473,4,1430.916667
2,235648,2747.000000,2713.000000,2679.000000,2671.000000,2652.000000,2631.000000,2589.000000,2450.000000,2144.000000,...,2457.000000,2482.000000,2452.000000,2450.000000,2449.0,2442.000000,-118.289893,33.794128,2,2304.291667
3,235649,1256.000000,1251.000000,1247.000000,1237.000000,1234.000000,1237.000000,1258.000000,1424.000000,1587.000000,...,1513.000000,1323.000000,1310.000000,1279.000000,1273.0,1254.000000,-118.304315,33.792617,2,1435.541667
4,235650,796.666667,794.666667,792.666667,790.000000,790.000000,816.000000,864.000000,1079.333333,1218.000000,...,823.333333,803.333333,798.666667,780.666667,786.0,789.333333,-118.247927,33.787470,3,991.250000


In [15]:
ambient_sp_id_df[['bid', 'ambient_avg']].to_sql('temptable3', engine, if_exists='replace', index=False)

In [16]:
sql = """
INSERT INTO ambient_population (bid, city, num_people) 
SELECT c.bid, 'LA', c.ambient_avg
FROM temptable3 c 
"""

result = engine.execute(text(sql))